# 천간(십간), 지지(십이지)와 음양오행의 관계 초기화

In [ ]:
heavenly_stems = {
    '갑': {'yang': 1,'yin': 0,'wood': 1,'fire': 0,'earth': 0,'metal': 0,'water': 0},
    '을': {'yang': 0,'yin': 1,'wood': 1,'fire': 0,'earth': 0,'metal': 0,'water': 0},
    '병': {'yang': 1,'yin': 0,'wood': 0,'fire': 1,'earth': 0,'metal': 0,'water': 0},
    '정': {'yang': 0,'yin': 1,'wood': 0,'fire': 1,'earth': 0,'metal': 0,'water': 0},
    '무': {'yang': 1,'yin': 0,'wood': 0,'fire': 0,'earth': 1,'metal': 0,'water': 0},
    '기': {'yang': 0,'yin': 1,'wood': 0,'fire': 0,'earth': 1,'metal': 0,'water': 0},
    '경': {'yang': 1,'yin': 0,'wood': 0,'fire': 0,'earth': 0,'metal': 1,'water': 0},
    '신': {'yang': 0,'yin': 1,'wood': 0,'fire': 0,'earth': 0,'metal': 1,'water': 0},
    '임': {'yang': 1,'yin': 0,'wood': 0,'fire': 0,'earth': 0,'metal': 0,'water': 1},
    '계': {'yang': 0,'yin': 1,'wood': 0,'fire': 0,'earth': 0,'metal': 0,'water': 1}
} # 천간

earthly_branches = {
    '자': {'yang': 1,'yin': 0,'wood': 0,'fire': 0,'earth': 0,'metal': 0,'water': 1},
    '축': {'yang': 0,'yin': 1,'wood': 0,'fire': 0,'earth': 1,'metal': 0,'water': 0},
    '인': {'yang': 1,'yin': 0,'wood': 1,'fire': 0,'earth': 0,'metal': 0,'water': 0},
    '묘': {'yang': 0,'yin': 1,'wood': 1,'fire': 0,'earth': 0,'metal': 0,'water': 0},
    '진': {'yang': 1,'yin': 0,'wood': 0,'fire': 0,'earth': 1,'metal': 0,'water': 0},
    '사': {'yang': 0,'yin': 1,'wood': 0,'fire': 1,'earth': 0,'metal': 0,'water': 0},
    '오': {'yang': 1,'yin': 0,'wood': 0,'fire': 1,'earth': 0,'metal': 0,'water': 0},
    '미': {'yang': 0,'yin': 1,'wood': 0,'fire': 0,'earth': 1,'metal': 0,'water': 0},
    '신': {'yang': 1,'yin': 0,'wood': 0,'fire': 0,'earth': 0,'metal': 1,'water': 0},
    '유': {'yang': 0,'yin': 1,'wood': 0,'fire': 0,'earth': 0,'metal': 1,'water': 0},
    '술': {'yang': 1,'yin': 0,'wood': 0,'fire': 0,'earth': 1,'metal': 0,'water': 0},
    '해': {'yang': 0,'yin': 1,'wood': 0,'fire': 0,'earth': 0,'metal': 0,'water': 1},
} # 지지

# 음양력 데이터 받아오기

In [ ]:
import requests
import time
import traceback
import pandas as pd
import lxml.etree as etree
from datetime import datetime, timedelta

In [ ]:
base_url = 'http://apis.data.go.kr/B090041/openapi/service/LrsrCldInfoService/getLunCalInfo'
service_key = '발급받은 서비스 키'

df = pd.DataFrame(columns = [
    'date',
    'yeonju',
    'wolju',
    'ilju',
    'siju',
    'yin',
    'yang',
    'wood',
    'fire',
    'water',
    'earth',
    'metal',
    'leap_month'
])
print(df)

session = requests.Session()

# 시작 날짜 조정
cur = datetime(2000, 4, 13)
# 추출할 날짜 개수 (공공 api 일별 최대 호출 제한 : 10000)
for i in range(0, 10000):
  if cur > datetime.now():
    break

  params = {
      'solYear': cur.year,
      'solMonth': "{:02d}".format(cur.month),
      'solDay': "{:02d}".format(cur.day),
      'ServiceKey': service_key,
  }

  url = base_url + '?' + '&'.join([f'{key}={value}' for key, value in params.items()])

  try:
    response = session.get(url)
    if response.status_code != 200:
      time.sleep(3)
      continue

    root = etree.fromstring(response.text.replace('<?xml version="1.0" encoding="UTF-8" standalone="yes"?>', ''))

    date = "{:04d}-{:02d}-{:02d}".format(cur.year, cur.month, cur.day)
    yeonju = root.xpath('//lunSecha/text()')
    wolju = root.xpath('//lunWolgeon/text()')
    ilju = root.xpath('//lunIljin/text()')
    leap_month = root.xpath('//lunLeapMonth/text()')

    is_leap_month = 0
    if leap_month == '윤':
      is_leap_month = 1

    yang, yin, wood, fire, water, earth, metal = 0, 0, 0, 0, 0, 0, 0
    for w in [yeonju, wolju, ilju]:
      if len(w) == 0:
        wolju.append('')
        continue

      hs, eb = w[0][0], w[0][1]
      yang += heavenly_stems[hs]['yang'] + earthly_branches[eb]['yang']
      yin += heavenly_stems[hs]['yin'] + earthly_branches[eb]['yin']
      wood += heavenly_stems[hs]['wood'] + earthly_branches[eb]['wood']
      fire += heavenly_stems[hs]['fire'] + earthly_branches[eb]['fire']
      water += heavenly_stems[hs]['water'] + earthly_branches[eb]['water']
      earth += heavenly_stems[hs]['earth'] + earthly_branches[eb]['earth']
      metal += heavenly_stems[hs]['metal'] + earthly_branches[eb]['metal']

    row = pd.DataFrame({'date': date, 'yeonju': yeonju, 'wolju': wolju, 'ilju': ilju, 'yang': yang, 'yin': yin, 'wood': wood, 'fire': fire, 'water': water, 'earth': earth, 'metal': metal, 'leap_month': is_leap_month})
    df = pd.concat([df, row], ignore_index=True)
  except Exception as e:
    print(f"예기치 않은 오류가 발생했습니다: {e}")
    print("cur date:", cur)
    print("yeonju:", yeonju)
    print("wolju:", wolju)
    print("ilju:", ilju)
    traceback.print_exc()
    break

  cur = cur + timedelta(days=1)
  time.sleep(0.25)

session.close()

print(df)
df.to_csv('../yinyang_calendar.csv', index=False)

Empty DataFrame
Columns: [date, yeonju, wolju, ilju, siju, yin, yang, wood, fire, water, earth, metal, leap_month]
Index: []
            date  yeonju   wolju    ilju siju yin yang wood fire water earth  \
0     2000-04-13  경진(庚辰)  경진(庚辰)  신축(辛丑)  NaN   2    4    0    0     0     3   
1     2000-04-14  경진(庚辰)  경진(庚辰)  임인(壬寅)  NaN   0    6    1    0     1     2   
2     2000-04-15  경진(庚辰)  경진(庚辰)  계묘(癸卯)  NaN   2    4    1    0     1     2   
3     2000-04-16  경진(庚辰)  경진(庚辰)  갑진(甲辰)  NaN   0    6    1    0     0     3   
4     2000-04-17  경진(庚辰)  경진(庚辰)  을사(乙巳)  NaN   2    4    1    1     0     2   
...          ...     ...     ...     ...  ...  ..  ...  ...  ...   ...   ...   
8926  2024-09-20  갑진(甲辰)  계유(癸酉)  정해(丁亥)  NaN   4    2    1    1     2     1   
8927  2024-09-21  갑진(甲辰)  계유(癸酉)  무자(戊子)  NaN   2    4    1    0     2     2   
8928  2024-09-22  갑진(甲辰)  계유(癸酉)  기축(己丑)  NaN   4    2    1    0     1     3   
8929  2024-09-23  갑진(甲辰)  계유(癸酉)  경인(庚寅)  NaN   2    4    2    0     1     